# Run SPM GLM to get condition beta weights and residual files

Loop over subject and TR, thereby concatenating across session

In [1]:
from bids.layout import BIDSLayout
from nipype.interfaces import afni 
from nipype.interfaces.io import BIDSDataGrabber, DataFinder, DataSink, DataGrabber
import nipype.pipeline as pe
import nipype as ni
from nipype.interfaces.utility import Function
import nipype.interfaces.fsl.maths as fsl
from nipype.interfaces import spm as spm
from nipype.algorithms import modelgen as mgen
from nipype.algorithms.misc import Gunzip 
import pandas as pd
import os, re, json
# https://nipype.readthedocs.io/en/0.11.0/users/spmmcr.html

matlab_cmd = '/opt/spm12-r7219/run_spm12.sh /opt/matlabmcr-2010a/v713/ script'
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

In [2]:
Basedir = "/scratch/qbi/uqkgarn1/STRIWP1/"
layout = BIDSLayout(Basedir)
subs = layout.get_subjects()

glm = pe.Workflow(name="glms") # workflow to run the analysis

/opt/miniconda-latest/envs/neuro/lib/python3.6/site-packages/bids/layout/models.py:102: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)


## Data grabbing and saving nodes

Note: I copied the motion files from the derivatives/etc path to derivatives/glm/sub-etc and I removed the T2 echo-1 and echo-2 .nii.gz data from the func folder below. I also removed the echo 2 motion files from the same folder, and the bjson files (renaming echo 1 of the 700 TR to fit the below field template

In [3]:
# THIS CODE DEFINITELY WORKS
dgT2s = pe.Node(DataGrabber(infields=['sub', 'TR'], 
                            outfields=['func','motion','onsets','bjson','mask']), name='T2-grabber')
dgT2s.inputs.base_dir = "/scratch/qbi/uqkgarn1/STRIWP1/"
dgT2s.inputs.sort_filelist = True
dgT2s.inputs.template='*'
dgT2s.inputs.template_args = {'func': [['sub', 'sub', 'TR']],
                              'motion':[['sub', 'sub', 'TR']],
                              'onsets':[['sub', 'sub', 'TR']],
                              'bjson':[['sub', 'sub', 'TR']],
                              'mask':[['sub', 'sub', 'TR']]}
dgT2s.inputs.field_template = {'func': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-%s/ses-*/func/sub-%s_*TR%s_space-T1w_desc-preproc_bold.nii.gz',
                               'motion': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-%s/ses-*/func/sub-%s_*-TR%s*desc-motion_regressors.txt',
                               'onsets': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-%s/ses-*/func/sub-%s_*-TR%s_glm_onsets.json',
                               'bjson': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-%s/ses-*/func/sub-%s_*-TR%s*space-T1w_desc-preproc_bold.json',
                               'mask': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-%s/ses-02/func/sub-%s_*TR%s_space-T1w_desc-brain_mask.nii.gz'}


In [4]:
# debugging
# dgT2s.inputs.sub = '01'
# dgT2s.inputs.TR = '700'
# res = dgT2s.run()
# res.outputs

In [5]:
# THIS WORKS
T2inf = pe.Node(ni.IdentityInterface(fields=['sub', 'TR']),
                   name='T2-info')
T2inf.iterables = [('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]
# T2inf.iterables = [('sub', ['03']), ('TR', ['1920'])]
T2inf.iterables 

[('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]

In [6]:
def printSubPath(fullFilePath):
    # function to split filepath into constituent parts, then print string to add as input to DataSink for the container string
    # given the full filepath, this extracts the subject folder and TR strings for input
    # into DataSink
    import os
    import re
    fname = os.path.normpath(fullFilePath[0])
    fname
    l = fname.split(os.sep)
    TR = re.search('.*acq-TR([0-9]*)_.*',l[-1])
    TR = str(int(TR.group(1)))
    name = [s for s in l if re.search('sub', s)][0]
    name = [name, "TR"+TR]
    name
    name = '/'.join(name)
    return name

# Data sink


In [7]:
ds = pe.Node(DataSink(), name='sink-stuff')
ds.inputs.base_directory = "/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/"
substitutions = [('_TR_([0-9]*)_sub_([0-9]*)', '')]
ds.inputs.regexp_substitutions = substitutions

# Get design info

In [8]:
# the input function should be a list of the prt files, taken across sessions, for each TR
def getOnsetsJson(input_files):
    from nipype.interfaces.base import Bunch
    import json
    prt_output = [] #prt=protocol
    count = 0
    for f in input_files:
        count = count + 1
        with open(f, "r+") as file:
            data = json.load(file)
            prt_output.insert(count, 
                              Bunch(conditions=data['names'],
                                    onsets=data['onsets'],
                                    durations=data['durations']))
    return prt_output

In [9]:
getOnsets = pe.Node(Function(input_names=['input_files'],
                             output_names=['prt_output'],
                             function=getOnsetsJson),
                    name='get_prt_onsets')

# Gunzip Nodes

1 for functional data, 1 for mask (condense during finesse)

In [10]:
gunzipfunc = pe.MapNode(Gunzip(), name='gunzipfunc', iterfield=['in_file'])

In [11]:
gmask = pe.Node(Gunzip(), name='m-zip')

# Smooth the functional data

In [ ]:
# smooth = pe.Node(spm.Smooth(), name='smoooooth')
# smooth.inputs.fwhm = [3, 3, 3]

# Get the TR for the model spec

In [12]:
# the input function should be a list of the prt files, taken across sessions, for each TR
def getTRJson(input_files):
    import json
    with open(input_files[0], "r+") as file:
            data = json.load(file)
            TR = data['RepetitionTime'] 
    if TR < .5:
        TR = 1.92
    return TR

In [13]:
getTR = pe.Node(Function(input_names=['input_files'],
                         output_names=['TR'],
                         function=getTRJson),
                name='get_TR')

In [14]:
def printSavFol(TR):
    # function to append the TR to a name for the save folder
    name = "TR" + TR
    return name

# Specify GLM Model

In [15]:
# SpecifyModel - Generates SPM-specific Model
modelspec = pe.Node(mgen.SpecifySPMModel(concatenate_runs=False,
                                         input_units='secs',
                                         output_units='secs',
                                         high_pass_filter_cutoff=128),
                    name="modelspec")

# Generate design matrix

In [16]:
# Level1Design - Generates an SPM design matrix
level1design = pe.Node(spm.Level1Design(bases={'hrf': {'derivs': [0, 0]}},
                                 timing_units='secs'),
                       name="level1design")

# Estimate model

In [17]:
# EstimateModel - estimate the parameters of the model
estimate = pe.Node(spm.EstimateModel(estimation_method={'Classical': 1},
                                     write_residuals=False),
                                     name="estimate")

# inputs
# spm mat file
# outputs
# beta_images
# residual_images
# spm_mat_file

# will input the spm.mat files output by the design module above

# # EstimateContrast - estimates contrasts
# level1conest = pe.Node(spm.EstimateContrast(), name="level1conest")
# level1conest.inputs.contrasts = contrast_list

### esimating beta weights and some contrasts due to below from Puckett's paper:

All functional time-courses were scaled to percent signal change (i.e.
each voxel time series was scaled to have a mean of 100) before calculating a multiple linear regression. The regression model contained 2
regressors corresponding to each experimental condition (simple and
complex movement, Fig. 1C) generated by convolving the stimulus
timing of each condition with a canonical hemodynamic response function model. A further 6 regressors, estimated from the volume registration step and representing the participant's head movement during the
scan, were also included. From the calculated regression model, contrast to-noise ratio (CNR) was calculated on a voxel-wise basis for each condition by dividing each voxel's beta or linear contrast value from the
general linear model analysis by the standard deviation of the residual
error of the time-series for that voxel. Results from the single-subject
regression analysis 

So I can just take each regressor and I will take the RMSE of the effect sizes (making negative positive) and divide that by the standard deviation of the residuals, within each region

# Specify GLM contrasts

In [18]:
names = ['att_left_5','att_left_8','att_right_5','att_right_8','ll','lh','hh','hl','exp_high','exp_low','unexp_high','unexp_low','right_hand','left_hand','invalid']
#len(names)
attend_lvr = ('att_lvr', 'T', names, [-1, -1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
attend_p = ('att_p', 'T', names, [-1, 1, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
attend_interaction = ('att_int', 'T', names, [1, -1, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
attend_cue = ('att_cue', 'F', [attend_lvr, attend_p, attend_interaction])
# attend_cue = ('att_cue', 'F', names, [[-1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#                                       [0, 0, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#                                       [1, -1, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

llvhh = ('llvhh', 'T', names, [0, 0, 0, 0, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
lhvhl = ('lhvhl', 'T', names, [0, 0, 0, 0, 0, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0])
relval = ('relval', 'T', names, [0, 0, 0, 0, 1, -1, -1, 1, 0, 0, 0, 0, 0, 0, 0])
value_cue = ('value_cue', 'F', [llvhh, lhvhl, relval])
# value_cue = ('value_cue', 'F', names, [[0, 0, 0, 0, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
#                                        [0, 0, 0, 0, 0, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
#                                        [0, 0, 0, 0, 1, -1, -1, 1, 0, 0, 0, 0, 0, 0, 0]])
hand = ('hand', 'T', names, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 0])
contrasts = [attend_lvr, attend_p, attend_interaction, attend_cue, llvhh, lhvhl, relval, value_cue, hand]
#contrasts

In [19]:
conts = pe.Node(spm.EstimateContrast(contrasts=contrasts), name='conts')

# Connect Workflow

In [20]:
# glm.connect([(T2inf, dgT2s, [('sub',  'sub')]), 
#              (T2inf, dgT2s, [('TR',   'TR')]),
#              (dgT2s, ds, [(('motion', printSubPath),
#                             'container')]),
#              (dgT2s, getOnsets, [('onsets', 'input_files')]),
#              (dgT2s, gunzipfunc, [('func', 'in_file')]),
#              (dgT2s, gmask, [('mask', 'in_file')]),
#              (gunzipfunc, smooth, [('out_file', 'in_files')]),
#              (dgT2s, getTR, [('bjson', 'input_files')]),
#              (getTR, modelspec, [('TR', 'time_repetition')]),
#              (dgT2s, modelspec, [('motion', 'realignment_parameters')]),
#              (getOnsets, modelspec, [('prt_output', 'subject_info')]),
#              (smooth, modelspec, [('smoothed_files', 'functional_runs')]),
#              (getTR, level1design, [('TR', 'interscan_interval')]),
#              (modelspec, level1design, [('session_info', 'session_info')]),
#              (gmask, level1design, [('out_file', 'mask_image')]),
#              (level1design, estimate, [('spm_mat_file', 'spm_mat_file')]),
#              (estimate, ds, [('beta_images', 'FLGLM.@beta')]),
#              (estimate, ds, [('residual_image', 'FLGLM.@resid')]),
#              (estimate, conts, [('beta_images', 'beta_images')]),
#              (estimate, conts, [('residual_image', 'residual_image')]),
#              (estimate, conts, [('spm_mat_file','spm_mat_file')]),
#              (conts, ds, [('con_images', 'FLGLM.@con')]),
#              (conts, ds, [('spmT_images', 'FLGLM.@T')]),
#              (conts, ds, [('spm_mat_file', 'FLGLM.@spm')])
#             ])      

In [21]:
glm.connect([(T2inf, dgT2s, [('sub',  'sub')]), 
             (T2inf, dgT2s, [('TR',   'TR')]),
             (dgT2s, ds, [(('motion', printSubPath),
                            'container')]),
             (dgT2s, getOnsets, [('onsets', 'input_files')]),
             (dgT2s, gunzipfunc, [('func', 'in_file')]),
             (dgT2s, gmask, [('mask', 'in_file')]),
             (dgT2s, getTR, [('bjson', 'input_files')]),
             (getTR, modelspec, [('TR', 'time_repetition')]),
             (dgT2s, modelspec, [('motion', 'realignment_parameters')]),
             (getOnsets, modelspec, [('prt_output', 'subject_info')]),
             (gunzipfunc, modelspec, [('out_file', 'functional_runs')]),
             (getTR, level1design, [('TR', 'interscan_interval')]),
             (modelspec, level1design, [('session_info', 'session_info')]),
             (gmask, level1design, [('out_file', 'mask_image')]),
             (level1design, estimate, [('spm_mat_file', 'spm_mat_file')]),
             (estimate, ds, [('beta_images', 'FLGLM.@beta')]),
             (estimate, ds, [('residual_image', 'FLGLM.@resid')]),
             (estimate, conts, [('beta_images', 'beta_images')]),
             (estimate, conts, [('residual_image', 'residual_image')]),
             (estimate, conts, [('spm_mat_file','spm_mat_file')]),
             (conts, ds, [('con_images', 'FLGLM.@con')]),
             (conts, ds, [('spmT_images', 'FLGLM.@T')]),
             (conts, ds, [('spm_mat_file', 'FLGLM.@spm')])
            ])      

In [22]:
glm.run()

200914-16:37:21,146 nipype.workflow INFO:
	 Workflow glms settings: ['check', 'execution', 'logging', 'monitoring']
200914-16:37:21,536 nipype.workflow INFO:
	 Running serially.
200914-16:37:21,539 nipype.workflow INFO:
	 [Node] Setting-up "glms.T2-grabber" in "/tmp/tmpu76d6nng/glms/_TR_1920_sub_05/T2-grabber".
200914-16:37:21,546 nipype.workflow INFO:
	 [Node] Running "T2-grabber" ("nipype.interfaces.io.DataGrabber")
200914-16:37:21,572 nipype.workflow INFO:
	 [Node] Finished "glms.T2-grabber".
200914-16:37:21,574 nipype.workflow INFO:
	 [Node] Setting-up "glms.get_TR" in "/tmp/tmpqo44ui51/glms/_TR_1920_sub_05/get_TR".
200914-16:37:21,580 nipype.workflow INFO:
	 [Node] Running "get_TR" ("nipype.interfaces.utility.wrappers.Function")
200914-16:37:21,586 nipype.workflow INFO:
	 [Node] Finished "glms.get_TR".
200914-16:37:21,586 nipype.workflow INFO:
	 [Node] Setting-up "glms.m-zip" in "/tmp/tmpya2bj2w2/glms/_TR_1920_sub_05/m-zip".
200914-16:37:21,592 nipype.workflow INFO:
	 [Node] Runni

200914-16:45:04,269 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM/_TR_1920_sub_05/beta_0019.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM//beta_0019.nii
200914-16:45:04,337 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM/_TR_1920_sub_05/beta_0020.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM//beta_0020.nii
200914-16:45:04,412 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM/_TR_1920_sub_05/beta_0021.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM//beta_0021.nii
200914-16:45:04,486 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM/_TR_1920_sub_05/beta_0022.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM//beta_0022.nii
200914-16:45:04,551 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/

200914-16:45:07,70 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM/_TR_1920_sub_05/beta_0055.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM//beta_0055.nii
200914-16:45:07,140 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM/_TR_1920_sub_05/beta_0056.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM//beta_0056.nii
200914-16:45:07,194 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM/_TR_1920_sub_05/beta_0057.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM//beta_0057.nii
200914-16:45:07,288 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM/_TR_1920_sub_05/beta_0058.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM//beta_0058.nii
200914-16:45:07,347 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/d

200914-16:45:08,901 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM/_TR_1920_sub_05/con_0006.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM//con_0006.nii
200914-16:45:08,905 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM/_TR_1920_sub_05/con_0007.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM//con_0007.nii
200914-16:45:08,909 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM/_TR_1920_sub_05/ess_0008.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM//ess_0008.nii
200914-16:45:08,970 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM/_TR_1920_sub_05/con_0009.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM//con_0009.nii
200914-16:45:08,975 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivati

200914-16:55:33,275 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1510/FLGLM/_TR_1510_sub_05/beta_0005.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1510/FLGLM//beta_0005.nii
200914-16:55:33,329 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1510/FLGLM/_TR_1510_sub_05/beta_0006.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1510/FLGLM//beta_0006.nii
200914-16:55:33,391 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1510/FLGLM/_TR_1510_sub_05/beta_0007.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1510/FLGLM//beta_0007.nii
200914-16:55:33,440 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1510/FLGLM/_TR_1510_sub_05/beta_0008.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1510/FLGLM//beta_0008.nii
200914-16:55:33,497 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/

...

200914-18:29:59,548 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/FLGLM/_TR_1510_sub_01/beta_0017.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/FLGLM//beta_0017.nii
200914-18:29:59,605 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/FLGLM/_TR_1510_sub_01/beta_0018.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/FLGLM//beta_0018.nii
200914-18:29:59,651 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/FLGLM/_TR_1510_sub_01/beta_0019.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/FLGLM//beta_0019.nii
200914-18:29:59,697 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/FLGLM/_TR_1510_sub_01/beta_0020.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/FLGLM//beta_0020.nii
200914-18:29:59,746 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/

200914-18:30:01,487 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/FLGLM/_TR_1510_sub_01/beta_0053.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/FLGLM//beta_0053.nii
200914-18:30:01,544 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/FLGLM/_TR_1510_sub_01/beta_0054.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/FLGLM//beta_0054.nii
200914-18:30:01,603 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/FLGLM/_TR_1510_sub_01/beta_0055.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/FLGLM//beta_0055.nii
200914-18:30:01,649 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/FLGLM/_TR_1510_sub_01/beta_0056.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/FLGLM//beta_0056.nii
200914-18:30:01,712 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/

200914-18:30:57,758 nipype.workflow INFO:
	 [Node] Finished "glms.level1design".
200914-18:30:57,759 nipype.workflow INFO:
	 [Node] Setting-up "glms.estimate" in "/tmp/tmpkw5n6qpe/glms/_TR_700_sub_01/estimate".
200914-18:30:57,772 nipype.workflow INFO:
	 [Node] Running "estimate" ("nipype.interfaces.spm.model.EstimateModel")
200914-18:36:20,946 nipype.workflow INFO:
	 [Node] Finished "glms.estimate".
200914-18:36:20,947 nipype.workflow INFO:
	 [Node] Setting-up "glms.conts" in "/tmp/tmpi6xskbbm/glms/_TR_700_sub_01/conts".
200914-18:36:21,26 nipype.workflow INFO:
	 [Node] Running "conts" ("nipype.interfaces.spm.model.EstimateContrast")
200914-18:36:32,751 nipype.workflow INFO:
	 [Node] Finished "glms.conts".
200914-18:36:32,753 nipype.workflow INFO:
	 [Node] Setting-up "glms.sink-stuff" in "/tmp/tmprbmxkvve/glms/_TR_700_sub_01/sink-stuff".
200914-18:36:32,773 nipype.workflow INFO:
	 [Node] Running "sink-stuff" ("nipype.interfaces.io.DataSink")
200914-18:36:32,775 nipype.interface INFO:


200914-18:36:33,982 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/FLGLM/_TR_700_sub_01/beta_0033.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/FLGLM//beta_0033.nii
200914-18:36:34,22 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/FLGLM/_TR_700_sub_01/beta_0034.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/FLGLM//beta_0034.nii
200914-18:36:34,51 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/FLGLM/_TR_700_sub_01/beta_0035.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/FLGLM//beta_0035.nii
200914-18:36:34,86 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/FLGLM/_TR_700_sub_01/beta_0036.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/FLGLM//beta_0036.nii
200914-18:36:34,129 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm

200914-18:36:35,71 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/FLGLM/_TR_700_sub_01/spmF_0004.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/FLGLM//spmF_0004.nii
200914-18:36:35,99 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/FLGLM/_TR_700_sub_01/spmT_0005.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/FLGLM//spmT_0005.nii
200914-18:36:35,141 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/FLGLM/_TR_700_sub_01/spmT_0006.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/FLGLM//spmT_0006.nii
200914-18:36:35,172 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/FLGLM/_TR_700_sub_01/spmT_0007.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/FLGLM//spmT_0007.nii
200914-18:36:35,214 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/gl